In [1]:
dataloc = r"C:\Users\ASUS\OneDrive\Desktop\code\datasets\CUAD_v1\CUAD_v1.json" 
import json
from datasets import Dataset


c:\Users\ASUS\.conda\envs\huggingface\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

with open(dataloc) as f:
    cuad = json.load(f)

In [12]:
cuad_entry = cuad['data'][0]['paragraphs'][0]

In [13]:
LABEL_MAP = {
    "Termination for Convenience": "termination",
    "Termination for Cause": "termination",
    "Confidentiality": "confidentiality",
    "Non-Disclosure": "confidentiality",
    "Limitation of Liability": "liability",
    "Governing Law": "governing_law",
    "Payment Terms": "payment_terms",
}

def extract_clause(context, start, window=300):
    s = max(0, start - window)
    e = min(len(context), start + window)
    return context[s:e].strip()

training_data = []

import re

def extract_label_from_question(question):
    match = re.search(r'"([^"]+)"', question)
    if match:
        return match.group(1)
    return None


for qa in cuad_entry["qas"]:
    clause_type = extract_label_from_question(qa["question"])

    if clause_type not in LABEL_MAP:
        continue

    label = LABEL_MAP[clause_type]

    for ans in qa["answers"]:
        clause_text = extract_clause(
            context=cuad_entry["context"],
            start=ans["answer_start"]
        )

        training_data.append({
            "text": clause_text,
            "label": label
        })



In [14]:
training_data

[{'text': 'other  party,  nor shall be subject to  injunctive  relief by the other          party if that party\'s  performance of its duties or  obligations  under          this  Agreement  is the  consequence  of Force  Majeure  as  defined in          Section 2.2 hereunder.\n\n6.9      Governing Law.  This Agreement is to be construed according to the laws          of the State of Illinois.\n\n7.       NEW PRODUCTS\n\n         7.1      Right of Option.  Should Company  introduce  other products or                   devices as contemplated by recital paragraph "A",  Distributor                   shall',
  'label': 'governing_law'}]